# Homework 1: Due February 13 at 3:00PM.
Fork/clone this repo and then solve the problems in this notebook. Make sure your code is well-commented, separated into different blocks in a smart way, and executed before your push it to **your** repository.

## Problem 1: Coding, integration, and a simple econ example
A profit-maximizing firm faces a demand curve given by: $P(q) = a-bq$ where $b\sim logN(\mu,\sigma)$ and has a cost function given by $C(q) = cq$. 

1. Solve for the optimal quantity analytically. Write your proof in the notebook using markdown.
2. Write a function called `profit_max_q(a, c, mu, sigma, method, n)` that returns the numerical optimal quantity given a set of inputs $(a, c, \mu, \sigma, method, n)$, where `method` is a string that takes on a value of `"mc"` or `"quad"` and determines whether you integrate using Monte Carlo or quadrature methods, and `n` is the number of Monte Carlo draws or quadrature nodes.
3. Choose a set of values $(a, c, \mu, \sigma)$ and use `profit_max_q` to solve the problem for both approaches to integration. Use the `CompEcon` package to implement the quadrature routine.

In [11]:
using CompEcon, Statistics

# Return optimal quantity
function quantity(a, c, mu, sigma, method, n)
    
    if method == :quad
        
        # expectation of b is weights' * nodes
        expected_b = qnwlogn(n,mu,sigma^2)[2]' * qnwlogn(n,mu,sigma^2)[1]
        
    elseif method == :mc
        
        # draw standard random, times SD, + mu, exponentiate to get logn variable
        mc_draws = exp.(randn(n) * sigma .+ mu)
        expected_b = mean(mc_draws)
        
    end
    
    # compute quantity using analytic solution
    q = (a - c) ./ (2 .* expected_b)[1]
    return q
    
end

quantity (generic function with 1 method)

In [20]:
inputs = [5., 3., 4., 3., :mc, 5000000]
println("The optimal quantity for input vector $(inputs[1:4]) using MC is $(quantity(inputs[1],inputs[2],inputs[3],inputs[4],inputs[5],inputs[6]))")
inputs = [5., 3., 4., 3., :quad, 11]
println("The optimal quantity for input vector $(inputs[1:4]) using quadrature is $(quantity(inputs[1],inputs[2],inputs[3],inputs[4],inputs[5],inputs[6]))")

The optimal quantity for input vector Any[5.0, 3.0, 4.0, 3.0] using MC is 0.00020627553071760947
The optimal quantity for input vector Any[5.0, 3.0, 4.0, 3.0] using quadrature is 0.0002034904748652255


## Problem 2: Coding and Monte Carlo
Approximate $\pi$ using Monte Carlo. You may only use `rand()` to generate random numbers. Here is how to think about approximating $\pi$:
1. Suppose $U$ is a two dimensional random variable on the unit square $[0,1]\times[0,1]$. The probability that $U$ is in a subset $B$ of $(0,1)\times(0,1)$ is equal to the area of $B$.
2. If $u_1,...,u_n$ are iid draws from $U$, then as $n$ grows (by an LLN type argument), the fraction that falls inside $B$ is the probability of another iid draw coming from $B$.
3. The area of a circle is given by $\pi \times radius^2$.

In [24]:
n = 1000000
count = 0


for i in 1:n
    
    # draw 2 random numbers
    u, v = rand(2)
    
    # Euclidean distance from middle of square
    d = sqrt((u - 0.5)^2 + (v - 0.5)^2) 
    
    # if inside the radius, add to count
    if d < 0.5
        count += 1
    end
    
end

# area is fraction of draws inside the radius, divide by r^2 to get π estimate
pi_estimate = count / n  / .5^2

println(pi_estimate)

3.144212


## Problem 3: Git

1. Create a new repository named `problem-set-1-q-3` on your GitHub account.
2. Put in a `README.md` with the following text: `Hello World!`.
3. Put in a .gitignore file, ignoring the Jupyter files .ipynb_checkpoints and the project files, .projects.
4. Create a new branch called `new-branch`.
5. Change the `README.md` text to `Goodbye World!`.
6. Merge `new-branch` back into `master`.

## Problem 4: Memory location

Let's learn about some of the nuances of memory allocation.

1. Generate one $20000 \times 20000$ array of random numbers named `x`. 
2. Make a function called `exp_cols` which exponentiates the elements of `x` column by column (i.e. by broadcasting `exp.()`) and returns the exponentiated array.
3. Make a function called `exp_rows` which exponentiates the elements of `x` row by row (i.e. by broadcasting `exp.()`) and returns the exponentiated array.
4. Call `exp_cols(x)` and `exp_rows(x)` twice and calculate the elapsed time on the second call (avoids fixed cost of initial compiliation).
5. Is one faster than the other?

In [31]:
random_array = rand(20000,20000)

function exp_col(random_array)
    
    # generate output array
    col_exp_array = similar(random_array)
    
    # loop and exponentiate
    for col_ind = 1:size(random_array, 2)
        
        col_exp_array[:, col_ind] = exp.(random_array[:, col_ind])
        
    end
    
    return col_exp_array
    
end

function exp_row(random_array)
    
    row_exp_array = similar(random_array)
    
    for row_ind = 1:size(random_array, 1)
        
        row_exp_array[row_ind, :] = exp.(random_array[row_ind, :])
        
    end
    
    return row_exp_array
    
end

# run and time both operations, run both twice because we need to pre-compile
exp_col(random_array)
@time col_exp_array = exp_col(random_array)

exp_row(random_array)
@time row_exp_array = exp_row(random_array)

# check whether the output is identical
all(row_exp_array .== col_exp_array)

  8.347195 seconds (80.01 k allocations: 8.944 GiB, 9.58% gc time)
 40.105899 seconds (80.01 k allocations: 8.944 GiB, 3.48% gc time)


true